In [3]:
import pandas as pd

# Load merged LLaMA 3.1 output
df = pd.read_csv("LLaMA3.1_70_Merged_Multilingual.csv")

# Check if all three predictions agree (fixing column names)
df["All_Agree"] = (
    (df["LLaMA3.1_EN"] == df["LLaMA3.1_FA"]) &
    (df["LLaMA3.1_EN"] == df["LLaMA3.1_IT"])
)

# Calculate agreement rate
agreement_rate = df["All_Agree"].mean() * 100
print(f"✅ Full agreement across EN, FA, IT: {agreement_rate:.2f}%")

✅ Full agreement across EN, FA, IT: 53.33%


In [4]:
df["Disagree"] = ~df["All_Agree"]
disagreements_by_domain = df[df["Disagree"]].groupby("Domain_EN").size()
print(disagreements_by_domain)


Domain_EN
Activism & Social Change    2
Arts & Culture              1
Business & Economics        3
Caregiving & Parenting      2
Education                   2
Healthcare                  1
Hospitality & Service       3
Journalism & Media          4
Law & Justice               4
Politics & Leadership       2
Religion & Spirituality     2
Science & Technology        2
dtype: int64


In [5]:
import pandas as pd

# Load the merged LLaMA 3.1 output
df = pd.read_csv("LLaMA3.1_70_Merged_Multilingual.csv")

# Add Disagreement_Type column (fixing column names)
df["Disagreement_Type"] = df.apply(lambda row: 
    "FA diverged" if row["LLaMA3.1_EN"] == row["LLaMA3.1_IT"] != row["LLaMA3.1_FA"] else
    "IT diverged" if row["LLaMA3.1_EN"] == row["LLaMA3.1_FA"] != row["LLaMA3.1_IT"] else
    "EN diverged" if row["LLaMA3.1_FA"] == row["LLaMA3.1_IT"] != row["LLaMA3.1_EN"] else
    "All different", axis=1
)

# Print breakdown
print(df["Disagreement_Type"].value_counts())



Disagreement_Type
All different    36
IT diverged      13
EN diverged       6
FA diverged       5
Name: count, dtype: int64


In [7]:
print("Option frequency in FA:")
print(df["LLaMA3.1_FA"].value_counts(normalize=True).round(2) * 100)

Option frequency in FA:
LLaMA3.1_FA
B    33.0
C    23.0
D    23.0
A    20.0
Name: proportion, dtype: float64


In [8]:
import pandas as pd

# Load your aligned LLaMA 3.1 results
df = pd.read_csv("LLaMA3.1_70_Merged_Multilingual.csv")

# Identify disagreement rows (not all 3 agree)
df["Disagreement"] = (df["LLaMA3.1_EN"] != df["LLaMA3.1_FA"]) | \
                     (df["LLaMA3.1_EN"] != df["LLaMA3.1_IT"]) | \
                     (df["LLaMA3.1_FA"] != df["LLaMA3.1_IT"])

# Export only rows with disagreement
df_disagreement = df[df["Disagreement"] == True]
df_disagreement.to_csv("LLaMA3.1_Disagreements.csv", index=False)
print(f"✅ Exported {len(df_disagreement)} disagreement cases to LLaMA3.1_Disagreements.csv")

# Count disagreement cases per domain
disagreement_by_domain = df_disagreement["Domain_EN"].value_counts().reset_index()
disagreement_by_domain.columns = ["Domain", "Disagreement_Count"]
disagreement_by_domain.to_csv("LLaMA3.1_Disagreement_By_Domain.csv", index=False)
print("✅ Saved domain-wise disagreement counts to LLaMA3.1_Disagreement_By_Domain.csv")

# Show top domains with most disagreement
print("\n📊 Top domains with most disagreement:")
print(disagreement_by_domain.head(10))

✅ Exported 28 disagreement cases to LLaMA3.1_Disagreements.csv
✅ Saved domain-wise disagreement counts to LLaMA3.1_Disagreement_By_Domain.csv

📊 Top domains with most disagreement:
                     Domain  Disagreement_Count
0        Journalism & Media                   4
1             Law & Justice                   4
2     Hospitality & Service                   3
3      Business & Economics                   3
4  Activism & Social Change                   2
5      Science & Technology                   2
6                 Education                   2
7     Politics & Leadership                   2
8    Caregiving & Parenting                   2
9   Religion & Spirituality                   2


In [11]:
import pandas as pd
from collections import Counter

# Load merged results file for LLaMA 3.1
df = pd.read_csv("LLaMA3.1_70_Merged_Multilingual.csv")

# Check full agreement
df["Full_Agreement"] = (df["LLaMA3.1_EN"] == df["LLaMA3.1_FA"]) & (df["LLaMA3.1_EN"] == df["LLaMA3.1_IT"])

# Classify type of disagreement
def classify_disagreement(row):
    choices = {row["LLaMA3.1_EN"], row["LLaMA3.1_FA"], row["LLaMA3.1_IT"]}
    if len(choices) == 1:
        return "All same"
    elif len(choices) == 3:
        return "All different"
    elif row["LLaMA3.1_EN"] != row["LLaMA3.1_FA"] and row["LLaMA3.1_EN"] == row["LLaMA3.1_IT"]:
        return "FA diverged"
    elif row["LLaMA3.1_EN"] != row["LLaMA3.1_IT"] and row["LLaMA3.1_EN"] == row["LLaMA3.1_FA"]:
        return "IT diverged"
    elif row["LLaMA3.1_EN"] != row["LLaMA3.1_FA"] and row["LLaMA3.1_FA"] == row["LLaMA3.1_IT"]:
        return "EN diverged"
    else:
        return "Other"

df["Disagreement_Type"] = df.apply(classify_disagreement, axis=1)

# Count full agreement
agreement_count = df["Full_Agreement"].sum()
total = len(df)
agreement_pct = agreement_count / total * 100

# Disagreement breakdown
disagreement_summary = df["Disagreement_Type"].value_counts().to_frame("count")

# Frequency of choices in Persian
fa_freq = df["LLaMA3.1_FA"].value_counts(normalize=True).rename("proportion").to_frame()

# Domain-wise disagreement
domain_disagreements = df[df["Disagreement_Type"] != "All same"]["Domain_EN"].value_counts().to_frame("count")

# Display everything
print("🔎 Full Agreement Rate:")
print(f"{agreement_count}/{total} prompts ({agreement_pct:.2f}%)\n")

print("🧩 Disagreement Types:")
print(disagreement_summary, "\n")

print("🗳️ Option Frequency in Persian (LLaMA3.1_FA):")
print(fa_freq, "\n")

print("🌍 Domain-wise Disagreements:")
print(domain_disagreements)




🔎 Full Agreement Rate:
32/60 prompts (53.33%)

🧩 Disagreement Types:
                   count
Disagreement_Type       
All same              32
IT diverged           13
EN diverged            6
FA diverged            5
All different          4 

🗳️ Option Frequency in Persian (LLaMA3.1_FA):
             proportion
LLaMA3.1_FA            
B              0.333333
C              0.233333
D              0.233333
A              0.200000 

🌍 Domain-wise Disagreements:
                          count
Domain_EN                      
Journalism & Media            4
Law & Justice                 4
Hospitality & Service         3
Business & Economics          3
Activism & Social Change      2
Science & Technology          2
Education                     2
Politics & Leadership         2
Caregiving & Parenting        2
Religion & Spirituality       2
Healthcare                    1
Arts & Culture                1


In [1]:
import pandas as pd
import numpy as np

def bootstrap_llama31_agreement_CI(filepath, prefix="LLaMA3.1", num_bootstrap=1000, confidence=0.95):
    df = pd.read_csv(filepath)
    en, fa, it = f"{prefix}_EN", f"{prefix}_FA", f"{prefix}_IT"
    df["Full_Agreement"] = (df[en] == df[fa]) & (df[en] == df[it])

    bootstrapped_agreements = []
    for _ in range(num_bootstrap):
        sample = df.sample(frac=1.0, replace=True)
        agreement_pct = sample["Full_Agreement"].mean() * 100
        bootstrapped_agreements.append(agreement_pct)

    lower_bound = np.percentile(bootstrapped_agreements, (1 - confidence) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_agreements, (1 + confidence) / 2 * 100)
    mean_agreement = np.mean(bootstrapped_agreements)

    print(f"Bootstrapped {int(confidence*100)}% CI for LLaMA 3.1 agreement:")
    print(f"Mean agreement: {mean_agreement:.2f}%")
    print(f"{int(confidence*100)}% Confidence Interval: [{lower_bound:.2f}%, {upper_bound:.2f}%]")

# Example usage:
bootstrap_llama31_agreement_CI("LLaMA3.1_70_Merged_Multilingual.csv")


Bootstrapped 95% CI for LLaMA 3.1 agreement:
Mean agreement: 53.33%
95% Confidence Interval: [41.67%, 65.00%]


In [2]:
import pandas as pd
import numpy as np

def bootstrap_llama31_disagreement_CI(filepath, prefix="LLaMA3.1", num_bootstrap=1000, confidence=0.95):
    df = pd.read_csv(filepath)
    en, fa, it = f"{prefix}_EN", f"{prefix}_FA", f"{prefix}_IT"

    # Disagreement is when not all three match
    df["Disagreement"] = (df[en] != df[fa]) | (df[en] != df[it]) | (df[fa] != df[it])

    bootstrapped_disagreements = []
    for _ in range(num_bootstrap):
        sample = df.sample(frac=1.0, replace=True)
        disagreement_pct = sample["Disagreement"].mean() * 100
        bootstrapped_disagreements.append(disagreement_pct)

    lower_bound = np.percentile(bootstrapped_disagreements, (1 - confidence) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_disagreements, (1 + confidence) / 2 * 100)
    mean_disagreement = np.mean(bootstrapped_disagreements)

    print(f"Bootstrapped {int(confidence*100)}% CI for LLaMA 3.1 disagreement:")
    print(f"Mean disagreement: {mean_disagreement:.2f}%")
    print(f"{int(confidence*100)}% Confidence Interval: [{lower_bound:.2f}%, {upper_bound:.2f}%]")

# Example usage
bootstrap_llama31_disagreement_CI("LLaMA3.1_70_Merged_Multilingual.csv")


Bootstrapped 95% CI for LLaMA 3.1 disagreement:
Mean disagreement: 46.25%
95% Confidence Interval: [33.33%, 60.00%]
